In [ ]:
import pandas as pd

normal_df = pd.read_csv("qa_dataset_normal-500.csv")
yo20_df = pd.read_csv("qa_dataset_20yo-500.csv")
yo65_df = pd.read_csv("qa_dataset_65yo-500.csv")

merged_df = normal_df.merge(
    yo20_df[["instruction", "input"]], on="input", suffixes=("_normal", "_20yo")
).merge(yo65_df[["instruction", "input"]], on="input")

merged_df = merged_df.rename(
    columns={
        "instruction_normal": "question_normal",
        "instruction_20yo": "question_20yo",
        "instruction": "question_65yo",
        "input": "context",
    }
)

merged_df[["context", "question_normal", "question_20yo", "question_65yo"]].to_csv(
    "aligned_evaluation_set.csv", index=False
)

print(
    merged_df[["context", "question_normal", "question_20yo", "question_65yo"]].head()
)

In [1]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
import time
from openai import OpenAI

In [2]:
load_dotenv("/Users/priyal/Documents/honours/.env/key.env")
openai.api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
with open("/Users/priyal/Documents/honours/thesis-B/finetuned_gpt4_normal_id.txt") as f:
    model_normal = f.read().strip()

with open("/Users/priyal/Documents/honours/thesis-B/finetuned_gpt4_20yo_id.txt") as f:
    model_20yo = f.read().strip()

with open("/Users/priyal/Documents/honours/thesis-B/finetuned_gpt4_65yo_id.txt") as f:
    model_65yo = f.read().strip()

In [4]:
df = pd.read_csv("/Users/priyal/Documents/honours/datasets/evaluation_set.csv").head(50)
df.head()

,Unnamed: 0,context,question_normal,question_20yo,question_65yo
0,0,"Ayurveda is a comprehensive, natural health ca...",Can Ayurvedic practices influence our genes an...,Can changing my lifestyle or diet with Ayurved...,Can Ayurveda practices like changing my diet o...
1,1,ETHNOPHARMACOLOGICAL RELEVANCE: Diabetes is a ...,How can integrating Ayurveda principles with m...,I've heard that diabetes is more than just hav...,I've been struggling with my diabetes manageme...
2,2,"Epilepsy, a chronic non-communicable disease o...",How can Ayurveda be used in managing and treat...,I've heard that Ayurveda might help with manag...,I've been having seizures and my current medic...
3,3,Coalescence of traditional medicine Ayurveda a...,Can the use of in silico technology help impro...,"So, can technology like computers and stuff be...",Can combining traditional Ayurvedic treatments...
4,4,BACKGROUND: Migraine is generally recognized a...,How can Ayurvedic medicine help in preventing ...,I get migraines a lot and normal treatments do...,I often suffer from migraines. Are there any n...


System prompts are avoided to ensure that the model's responses reflect the effects of fine-tuning, not the pre-trained model’s behavior.

In [5]:
def get_response(model_id, context, question):
    prompt = f"Context:\n{context}\n\nQuestion:\n{question}"
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return "Error"

In [6]:
normal_outputs = []
yo20_outputs = []
yo65_outputs = []

for idx, row in df.iterrows():
    print(f"Processing row {idx+1}/50...")

    normal_outputs.append(
        get_response(model_normal, row["context"], row["question_normal"])
    )
    yo20_outputs.append(get_response(model_20yo, row["context"], row["question_20yo"]))
    yo65_outputs.append(get_response(model_65yo, row["context"], row["question_65yo"]))

    time.sleep(1.2)

Processing row 1/50...
Processing row 2/50...
Processing row 3/50...
Processing row 4/50...
Processing row 5/50...
Processing row 6/50...
Processing row 7/50...
Processing row 8/50...
Processing row 9/50...
Processing row 10/50...
Processing row 11/50...
Processing row 12/50...
Processing row 13/50...
Processing row 14/50...
Processing row 15/50...
Processing row 16/50...
Processing row 17/50...
Processing row 18/50...
Processing row 19/50...
Processing row 20/50...
Processing row 21/50...
Processing row 22/50...
Processing row 23/50...
Processing row 24/50...
Processing row 25/50...
Processing row 26/50...
Processing row 27/50...
Processing row 28/50...
Processing row 29/50...
Processing row 30/50...
Processing row 31/50...
Processing row 32/50...
Processing row 33/50...
Processing row 34/50...
Processing row 35/50...
Processing row 36/50...
Processing row 37/50...
Processing row 38/50...
Processing row 39/50...
Processing row 40/50...
Processing row 41/50...
Processing row 42/50...
P

In [7]:
df["gpt4_normal_response"] = normal_outputs
df["gpt4_20yo_response"] = yo20_outputs
df["gpt4_65yo_response"] = yo65_outputs

df.to_csv(
    "/Users/priyal/Documents/honours/datasets/gpt4_persona_responses_50.csv",
    index=False,
)
print("Responses saved to gpt4_persona_responses_50.csv ✅")

Responses saved to gpt4_persona_responses_50.csv ✅
